In [26]:
from apr_and_tvl_by_destination_script import _fetch_APY_and_allocation_data
from mainnet_launch.constants import ALL_AUTOPOOLS, AUTO_USD
import plotly.express as px

dfs = {a.name: _fetch_APY_and_allocation_data(a) for a in ALL_AUTOPOOLS}

In [40]:
import plotly.io as pio

pio.templates.default = None

# my judgement is that the expected return method is more reasonable, and less noisy,
# so during warmup I will display the expected return method

for a in ALL_AUTOPOOLS:
    autopool_name = a.name

    apr_df, percent_tvl_by_destination, expected_return_series, df = dfs[autopool_name]

    # baseUSD, sonicUSD and autoDOLA do not have autopoolAPY fields, so instead we use the 30 day moving average of autopoolDay30MAApy
    # this is because (to validate) they don't back out price return

    if df["autopoolApy"].isna().all():
        primary_display_apy = "autopoolDay30MAApy"
    else:
        primary_display_apy = "autopoolApy"

    df["display_apy_first_expected_return_then_UI_APY"] = df["expected_return"].where(
        df[primary_display_apy].isna(), df[primary_display_apy]
    )
    df["total_display_apy_expected_return"] = df["display_apy_first_expected_return_then_UI_APY"] + df["rewarderApy"]

    # my judgement is tha
    px.bar(percent_tvl_by_destination, title=f"{autopool_name} % Allocation Over Time").show()
    px.line(
        df[
            [
                "total_display_apy_expected_return",
            ]
        ],
        title=f"{autopool_name} APY (expected return method vs 7 day MA method)",
    ).show()

# todo include markdown description here

In [ ]:
# if base Asest in
df["autopoolApy"]

datetime
2025-05-30 00:00:00+00:00    NaN
2025-05-31 00:00:00+00:00    NaN
2025-06-01 00:00:00+00:00    NaN
2025-06-02 00:00:00+00:00    NaN
2025-06-03 00:00:00+00:00    NaN
2025-06-04 00:00:00+00:00    NaN
2025-06-05 00:00:00+00:00    NaN
2025-06-06 00:00:00+00:00    NaN
2025-06-07 00:00:00+00:00    NaN
2025-06-08 00:00:00+00:00    NaN
2025-06-09 00:00:00+00:00    NaN
2025-06-10 00:00:00+00:00    NaN
2025-06-11 00:00:00+00:00    NaN
2025-06-12 00:00:00+00:00    NaN
2025-06-13 00:00:00+00:00    NaN
2025-06-14 00:00:00+00:00    NaN
2025-06-15 00:00:00+00:00    NaN
2025-06-16 00:00:00+00:00    NaN
2025-06-17 00:00:00+00:00    NaN
2025-06-18 00:00:00+00:00    NaN
2025-06-19 00:00:00+00:00    NaN
2025-06-20 00:00:00+00:00    NaN
2025-06-21 00:00:00+00:00    NaN
2025-06-22 00:00:00+00:00    NaN
2025-06-23 00:00:00+00:00    NaN
2025-06-24 00:00:00+00:00    NaN
2025-06-25 00:00:00+00:00    NaN
2025-06-26 00:00:00+00:00    NaN
2025-06-27 00:00:00+00:00    NaN
2025-06-28 00:00:00+00:00    NaN
2

In [ ]:
# sonic does not have vlaues for
# autopoolDay30MAApy

In [33]:
df

,DOLA+scrvUSD (curve),DOLA/USR (curve),DOLA/sUSDS (curve),DOLA/sUSDe (curve),SaveDola (curve),Staked Dola (inverse),alUSD/sDOLA (curve),autoDOLA (tokemak),deUSD/DOLA (curve),reUSD/sDola (curve),...,autopoolDay1MAApy,autopoolDay7MABackedApy,autopoolDay30MABackedApy,autopoolApy,rewarderApy,expected_return,display_apy_first_7_day_then_UI_APY,total_display_apy_7_day_ma,display_apy_first_expected_return_then_UI_APY,total_display_apy_expected_return
datetime,,,,,,,,,,,,,,,,,,,,,
2025-05-30 00:00:00+00:00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+02,0.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN
2025-05-31 00:00:00+00:00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+02,0.0,0.000000,...,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,NaN
2025-06-01 00:00:00+00:00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,5.103798e+01,0.0,48.962016,...,0.000000,NaN,NaN,NaN,NaN,6.354033,NaN,NaN,6.354033,NaN
2025-06-02 00:00:00+00:00,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,9.711998e+01,0.0,2.880019,...,0.000000,NaN,NaN,NaN,1.306766e+02,0.381630,NaN,NaN,0.381630,131.058270
2025-06-03 00:00:00+00:00,0.0,0.0,0.000000,0.000000,0.000000,68.761444,0.000000,3.817928e+00,0.0,27.420627,...,78.457918,NaN,NaN,NaN,1.074392e+01,8.314145,NaN,NaN,8.314145,19.058068
2025-06-04 00:00:00+00:00,0.0,0.0,0.000000,0.000000,9.407893,53.970697,0.000000,1.058566e+01,0.0,26.035747,...,0.000000,NaN,NaN,NaN,5.536473e+00,7.633205,NaN,NaN,7.633205,13.169678
2025-06-05 00:00:00+00:00,0.0,0.0,0.000000,4.270788,15.765902,48.087409,0.000000,9.859035e-05,0.0,31.875802,...,43.313777,14.750031,NaN,NaN,4.923858e+00,8.259914,13.849468,18.773326,8.259914,13.183772
2025-06-06 00:00:00+00:00,0.0,0.0,1.914223,4.254877,15.704992,44.767880,0.000000,3.423203e-03,0.0,33.354604,...,0.000000,12.385019,NaN,NaN,4.961578e+00,8.597678,12.973148,17.934726,8.597678,13.559256
2025-06-07 00:00:00+00:00,0.0,0.0,1.906182,4.236938,17.576796,43.033577,0.000000,5.603026e-21,0.0,33.246507,...,35.140096,21.761367,NaN,NaN,6.415045e+00,8.607781,17.993161,24.408207,8.607781,15.022827
